In [7]:
import pandas as pd
import json
import glob
import os
import pandasql as ps

In [4]:
arquivos = glob.glob("*.txt")

linhas = []
for arquivo in arquivos:
    with open(arquivo, "r") as f:
        for linha in f:
            if linha.strip():  # ignora linhas vazias
                dados = json.loads(linha)  # cada linha é uma lista JSON
                for d in dados:
                    d["arquivo_origem"] = os.path.basename(arquivo)  # opcional: guarda nome do arquivo
                    linhas.append(d)
df = pd.json_normalize(linhas)
print(df.head())

# df.to_csv("dados_unificados.csv", index=False)

                  timestamp     type           mac  gatewayFree  gatewayLoad  \
0  2025-08-22T12:59:19.421Z  Gateway  AC233FC15CC5         57.0         0.07   
1  2025-08-22T12:59:19.614Z  Unknown  C30000400862          NaN          NaN   
2  2025-08-22T12:59:19.957Z  Unknown  C30000400861          NaN          NaN   
3  2025-08-22T12:59:20.209Z  Unknown  C30000400862          NaN          NaN   
4  2025-08-22T12:59:20.510Z  Gateway  AC233FC15CC5         57.0         0.07   

       seq               arquivo_origem  \
0  81488.0  2025-08-22_12-59-20.462.txt   
1      NaN  2025-08-22_12-59-20.462.txt   
2      NaN  2025-08-22_12-59-20.462.txt   
3      NaN  2025-08-22_12-59-20.462.txt   
4  81489.0  2025-08-22_12-59-20.462.txt   

                                             rawData bleName  bleNo  rssi  \
0                                                NaN     NaN    NaN   NaN   
1  02010617FF3906CA03246208400000C321642500082201...            2.0 -53.0   
2  02010617FF3906CA0324610840

In [13]:
df.info

<bound method DataFrame.info of                        timestamp     type           mac  gatewayFree  \
0       2025-08-22T12:59:19.421Z  Gateway  AC233FC15CC5         57.0   
1       2025-08-22T12:59:19.614Z  Unknown  C30000400862          NaN   
2       2025-08-22T12:59:19.957Z  Unknown  C30000400861          NaN   
3       2025-08-22T12:59:20.209Z  Unknown  C30000400862          NaN   
4       2025-08-22T12:59:20.510Z  Gateway  AC233FC15CC5         57.0   
...                          ...      ...           ...          ...   
367887  2025-08-23T01:33:57.963Z  Gateway  AC233FC15CC5         46.0   
367888  2025-08-23T01:33:58.558Z  Unknown  C30000400861          NaN   
367889  2025-08-23T01:33:58.860Z  Unknown  C30000400862          NaN   
367890  2025-08-23T01:33:59.033Z  Gateway  AC233FC15CC5         46.0   
367891  2025-08-23T01:33:59.607Z  Unknown  C30000400857          NaN   

        gatewayLoad       seq               arquivo_origem  \
0              0.07   81488.0  2025-08-22

In [18]:
pd.set_option("display.max_colwidth", None)

query = """
SELECT 
    timestamp, mac, rawData
FROM df
WHERE
   mac = 'C300001DA91C'
   OR mac = 'C300001DA912'
   OR mac = 'C3000049D100'
ORDER BY
    timestamp DESC
"""

result = ps.sqldf(query, locals())
print(result.head(10))
result.to_csv("raw_weekend.csv", index = False)

                  timestamp           mac  \
0  2025-08-23T21:58:44.663Z  C300001DA91C   
1  2025-08-23T21:58:44.562Z  C300001DA91C   
2  2025-08-23T21:58:44.360Z  C300001DA91C   
3  2025-08-23T21:58:44.259Z  C300001DA91C   
4  2025-08-23T21:58:44.107Z  C300001DA91C   
5  2025-08-23T21:58:44.107Z  C300001DA91C   
6  2025-08-23T21:58:43.814Z  C300001DA91C   
7  2025-08-23T21:57:48.256Z  C300001DA912   
8  2025-08-23T21:57:48.155Z  C300001DA912   
9  2025-08-23T21:57:48.064Z  C300001DA912   

                                                rawData  
0  0201060303E1FF1216E1FFA103640007FFF600F31CA91D0000C3  
1  0201060303E1FF1216E1FFA103640007FFFB00FA1CA91D0000C3  
2  0201060303E1FF1216E1FFA103640005FFF400FA1CA91D0000C3  
3  0201060303E1FF1216E1FFA103640007FFFE01001CA91D0000C3  
4  0201060303E1FF1216E1FFA103640002FFF400FA1CA91D0000C3  
5  0201060303E1FF1216E1FFA103640002FFF100F51CA91D0000C3  
6  0201060303E1FF1216E1FFA103640007FFF600F81CA91D0000C3  
7  0201060303E1FF1216E1FFA1036400050002F